# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲームコンペティションに向けたエージェントの構築を目的としています。特定のユーザー入力に応じて質問を生成したり、答えを提供したりするための言語モデル（LLM）を使用しています。このプロジェクトは、「Gemma」という事前学習済みモデルを利用しており、特にマルチターンの対話における情報の取得と推測に焦点を当てています。

### 提出物の内容
1. **環境設定とライブラリのインストール**:
   - Tensor Processing Unit (TPU) と Graphics Processing Unit (GPU) のセットアップをしています。
   - `immutabledict`と`sentencepiece`ライブラリおよび`gemma_pytorch`リポジトリがインストールされています。

2. **エージェントのクラス設計**:
   - `GemmaFormatter`, `GemmaAgent`, `GemmaQuestionerAgent`, `GemmaAnswererAgent`などのクラスが定義されており、これにより質問者と回答者の役割を担うエージェントが実装されています。
   - 各エージェントは、ユーザーの入力を受け取り、内部状態を保持し、適切な応答を生成します。

3. **応答生成の方法**:
   - パラメータ調整（例：temperature、top_p、top_k）を用いて生成プロセスを制御し、確率的に応答を生成します。
   - 正しいキーワードを推測するための解析機能も含まれています。

4. **テストと実行**:
   - Kaggle環境で20の質問ゲームをシミュレーションするために、`kaggle_environments`ライブラリを用いています。

5. **エージェントの相互作用**:
   - 質問者と回答者のターンを管理し、質問を交互に生成し合う仕組みが実装されています。

### 使用されているライブラリ
- **PyTorch**: 学習モデルの実行と管理に利用。
- **Gemma**: 特定の事前学習モデルを使用するためのライブラリとして活用。
- **kaggle_environments**: Kaggleの環境をシミュレーションし、エージェントのパフォーマンスを確認するために使用。

このノートブックは、「20の質問」ゲームの戦略的なプレイをサポートするためのインフラを整備しており、特に言語モデルがどのように自動的に質問し、回答し、ゲームを進行させるかに注力しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の解説を示します。

1. **immutabledict**:
   - Pythonの辞書（dict）と似ていますが、変更ができない（immutable）特性を持つデータ構造です。この特性により、辞書の内容を誤って変更することを防ぎ、ハッシュ可能なオブジェクトとして使用することができます。

2. **sentencepiece**:
   - 自然言語処理におけるテキストのトークナイゼーションのためのアルゴリズムです。文章をサブワードやトークンに分割することで、より柔軟なモデルの学習を可能にします。特に、形態素解析を必要としない言語処理に役立ちます。

3. **TPU（Tensor Processing Unit）**:
   - Googleが開発した特化型ハードウェアで、特にディープラーニングモデルのトレーニングや推論を高速化するために設計されています。一般的なGPUよりも効率的に行列演算を行うことができます。

4. **torch.FloatTensor**:
   - PyTorchのデータ構造の一つで、浮動小数点数のテンソルを表します。モデルの入力データやモデルの重みを格納するために使用されます。

5. **GemmaForCausalLM**:
   - Gemmaライブラリの一部で、因果言語モデル（Causal Language Model）を実装したクラスです。因果モデルは、過去の情報に基づいて次の単語を予測するために使用されます。

6. **prompt**:
   - モデルがテキストを生成するための入力文や形態のことです。システムプロンプトはモデルの出力を導くための指示となります。

7. **context manager**:
   - Pythonの構文の一つで、特定のオブジェクトを一時的に取り扱う際に、そのオブジェクトが自動的に破棄されるようにすることができます。省メモリ化や、リソースの管理に役立ちます。

8. **interleave**:
   - 二つのリストやシーケンスを交互に結合する操作のことです。異なるデータのペアを作成する際に使われます。

9. **sampler_kwargs**:
   - モデルの出力生成時に使用するための追加的な引数です。例えば、生成するテキストの多様性を制御するためのパラメータ（temperature、top_pなど）が含まれます。

10. **re.search**:
   - 正規表現を用いて文字列の検索を行うための関数です。特定のパターンが文字列内に存在するかをチェックし、結果として見つかった部分を返します。

11. **floatとdouble**:
   - 数値型の一つで、floatは単精度（32ビット）を、doubleは倍精度（64ビット）を表します。計算の精度に影響を与えます。

12. **Kaggleエージェント**:
   - Kaggleプラットフォームで実行されるエージェントプログラムで、特定の課題を解決するための戦略的なアルゴリズムを実装します。この場合、20の質問ゲームにおける質問者や回答者の役割を担います。

これらの用語は、機械学習の具体的な実装やコードの理解において重要な概念であり、特に実務未経験の初心者にとっては馴染みが薄いかもしれません。理解を深めるために、それぞれの実際の使用例や関連性をさらに調べると良いでしょう。

---


In [ ]:
%%bash
# 現在の作業ディレクトリを/kaggle/workingに移動する
cd /kaggle/working
# immutabledictとsentencepieceパッケージをインストールする
# -q:出力を最小限にする、-U:アップグレード、-t:指定したディレクトリにインストール
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorchリポジトリをGitHubからクローンする（出力は非表示にする）
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemma用のディレクトリを作成する
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorchリポジトリからgemmaのファイルを新しく作成したディレクトリに移動する
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
# TPU（Tensor Processing Unit）用の環境設定を行うためのスクリプトをダウンロードする
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# ダウンロードしたスクリプトを実行し、必要なAPTパッケージ（libomp5とlibopenblas-dev）をインストールする
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
# GPU（Graphics Processing Unit）用のデバイスを設定する
# デバイスがCUDA（GPU）を利用可能であればcuda:0を使用し、それ以外の場合はCPUを使用する
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TPU（Tensor Processing Unit）用のデバイスを設定する
device = xm.xla_device()
# デフォルトのテンソルタイプをtorch.FloatTensorに設定する
torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re
from typing import Iterable
from pathlib import Path

# Kaggleエージェントのパスを定義する
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
# Kaggleエージェントのパスが存在する場合
if os.path.exists(KAGGLE_AGENT_PATH):
    # ライブラリのパスをシステムパスに追加する
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    # 重みのパスを設定する
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    # その他の状況では、別のライブラリのパスを設定する
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

# gemmaライブラリから設定とモデルをインポートする
from gemma.config import get_config_for_2b
from gemma.model import GemmaForCausalLM

# GemmaFormatterクラスの定義
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()
    def __repr__(self):
        return self._state
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    def apply_turns(self, turns: Iterable, start_agent: str):
        # 質問者と回答者のターンを交互に適用する
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    # デフォルトのテンソルタイプを設定するためのコンテキストマネージャ
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

# GemmaAgentクラスの定義
class GemmaAgent:
    def __init__(self, sp=None, fse=None):
        self._device = xm.xla_device()  # デバイスをTPUに設定する
        self.formatter = GemmaFormatter(sp=sp, fse=fse)  # フォーマッタを初期化
        print("モデルを初期化中")
        model_config = get_config_for_2b()  # モデルの設定を取得
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'  # トークナイザーのパスを設定
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)  # モデルを初期化
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')  # 重みをロード
            self.model = model.to(self._device).eval()  # モデルを指定したデバイスに移動し評価モードにする
    def __call__(self, obs, *args):
        self._start_session(obs)  # セッションを開始
        prompt = str(self.formatter)  # プロンプトとしてフォーマッタの状態を取得
        response = self._call_llm(prompt)  # LLMを呼び出し
        response = self._parse_response(response, obs)  # 応答を解析
        print(f"{response=}")  # 応答を出力
        return response
    def _start_session(self, obs: dict):
        raise NotImplementedError  # サブクラスで実装すべきメソッド
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        # LLMを呼び出して応答を生成する
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.8, 'top_p': 0.9, 'top_k': 60,}
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response
    def _parse_keyword(self, response: str):
        # 応答からキーワードを解析する
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None: 
            keyword = ''
        else: 
            keyword = match.group().lower()
        return keyword
    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError  # サブクラスで実装すべきメソッド

# 2つのイテラブルを交互に結合する関数
def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

# GemmaQuestionerAgentクラスの定義
class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に格納
        self.formatter.apply_turns(turns, start_agent='model')  # フォーマッタにターンを適用
        # ターンのタイプに応じたプロンプトを設定する
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("キーワードを推測してください。推測を二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        # 応答を解析する
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: 
                question = "それは人ですか？"  # 人、場所、物の中からランダムに選択
            else: 
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else: 
            raise ValueError("不明なターンタイプ:", obs.turnType)

# GemmaAnswererAgentクラスの定義
class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}でカテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に格納
        self.formatter.apply_turns(turns, start_agent='user')  # フォーマッタにターンを適用
        self.formatter.user(f"質問はキーワード{obs.keyword}に関するものです。はいかいいえで答えて、答えを二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# システムプロンプトと例のセットを定義する
sp = "あなたは20の質問ゲームをプレイしています。このゲームでは、回答者がキーワードを考え、質問者のはい・いえの質問に答えます。キーワードは特定の人、場所、または物です。"
fse = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** キーワードを推測してください。",
    "**フランス**", "正解です！",
]
agent = None

# エージェントを取得する関数
def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "エージェントが初期化されていません。"
    return agent

def agent_fn(obs, cfg):
    # エージェントの応答を取得する
    if obs.turnType == "ask": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": 
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1: 
        return "はい"
    else: 
        return response

# テスト

In [ ]:
# pygameライブラリをインストールする
!pip install -q pygame

In [ ]:
# Kaggleの環境を作成するために必要なライブラリをインポートする
from kaggle_environments import make
# 20の質問ゲームのKaggle環境を作成する
env = make("llm_20_questions")

In [ ]:
# コードを実行する
%run submission/main.py

In [ ]:
# *** 修正が必要です ****
# 環境で質問者エージェントとランダムエージェントを実行する
# env.run([get_agent('questioner'), "random"])

In [ ]:
# 環境の状態をIPythonモードで表示する
env.render(mode="ipython")

# パッケージ

In [ ]:
# pigzとpvパッケージをインストールする（出力は非表示にする）
!apt install pigz pv > /dev/null

In [ ]:
# pigzを使用して、submissionディレクトリを圧縮し、進行状況を表示しながらtar.gzファイルを作成する
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2